## Завантаження моделей і вагів, та імпорт бібліотек

In [1]:
from tensorflow.keras.utils import load_img, img_to_array
# from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.image import resize
from keras.models import load_model
import numpy as np
import skimage
import cv2
import os
import re

## Тестові фото

In [181]:
dir_path = r'C:\Users\kuzik\Desktop\photos'
list_files = os.listdir(dir_path)
list_files
files  = []
for filename in list_files:
    files.append(re.sub(r'\\', r'/', dir_path + "/" + filename))

## Функція обробки фото і передбачення

In [153]:
def load_image(filename):
    img = load_img(filename)
    return img

def tf_resize(img):
    img = resize(img, (32, 32))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = np.copy(img)                   # Withot copy doesn't work on my PC. Cheсk after deploy.
    img = preprocess_input(img).astype("float32") / 255
    return img

def cv2_resize(img):
    img = cv2.resize(img, (32, 32), interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = np.copy(img)                   # Withot copy doesn't work on my PC. Cheсk after deploy.
    img = preprocess_input(img).astype("float32") / 255
    return img

def si_resize(img):
    img = skimage.transform.resize(img, (32, 32), anti_aliasing=True)
    img = np.expand_dims(img, axis=0)
    img = np.copy(img)                   # Withot copy doesn't work on my PC. Cheсk after deploy.
#     img = preprocess_input(img).astype("float32") / 255
    return img

def si_rescale(img):
    for_rescale = list(img.shape)
    for_rescale.sort()
    scale_rating = 32 / for_rescale[1]
    img = skimage.transform.rescale(skimage.transform.resize(img, (for_rescale[1], for_rescale[1])), scale_rating, multichannel=True)
    img = np.expand_dims(img, axis=0)
    img = np.copy(img)                   # Withot copy doesn't work on my PC. Cheсk after deploy.
#     img = preprocess_input(img).astype("float32") / 255
    return img

def image_classify(filename, processor, model):
    if processor == 'tf':
        img = load_image(filename)
        img = tf_resize(img)
    elif processor == 'cv2':
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        img = cv2_resize(img)
    elif processor == 'si_resize':
        img = skimage.io.imread(filename)
        img = si_resize(img)
    elif processor == 'si_rescale':
        img = skimage.io.imread(filename)
        img = si_rescale(img)
        
    features = model.predict(img)
    top = []
    top_num = 3
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    for request in range(1):  # num of foto or objects on foto
        classes_request = classes
        features_request = features[0]
        rang = 1
        for i in range(top_num):
            pred = np.argmax(features_request)
            top.append([rang, classes_request[pred], round(features_request[np.argmax(features_request)], 4)])
            features_request = np.delete(features_request, np.argmax(features_request))
            classes_request.remove(classes_request[pred])
            rang +=1
    return top

    

## Перевірка роботи моделі з стисканням від tensorflow та cv2

In [149]:
files = ['C:/Users/kuzik/Desktop/airplane1.jpeg', 'C:/Users/kuzik/Desktop/automobile1.jpg', 'C:/Users/kuzik/Desktop/bird1.jfif', 
         'C:/Users/kuzik/Desktop/cat1.jfif', 'C:/Users/kuzik/Desktop/deer1.jfif', 'C:/Users/kuzik/Desktop/dog1.jfif', 
         'C:/Users/kuzik/Desktop/frog1.jfif', 'C:/Users/kuzik/Desktop/horse1.jfif', 'C:/Users/kuzik/Desktop/ship1.jfif', 
         'C:/Users/kuzik/Desktop/truck1.jfif']

def obj_on_file(file):
    file_name = file.split("/")[-1]
    file_name = file_name.split(".")[0:-1]
    find = re.match(r'\D+', str(file_name)).group()[2:]
    return find

def testing(path, processor, model, print_result = False):
    result = []
    obj = []
    for file in path:
        o = obj_on_file(file)
        predict_img = image_classify(file, processor, model = model)
        obj.append(o)
        result.append(predict_img[0][1])
        
        if print_result == True:
            print("")
            print("Must be: ", obj)
            print(predict_img)
        
    return result, obj

def accuracy_of_model(path, processor, model, print_result = False):
    result_test, answer_test = testing(path = path, processor = processor, model = model, print_result = print_result)
    list_of_results = []
    for num in range(len(result_test)):
        if result_test[num] == answer_test[num]:
            list_of_results.append(1)
        else:
            list_of_results.append(0)
    return list_of_results
    

def compare_processors(path, model, processor=[], print_result=False):
    result = []
    for el in processor:
        model_accuracy = accuracy_of_model(path, model = model, processor = el, print_result = print_result)
        accuracy_percent = model_accuracy.count(1) / len(model_accuracy) * 100
        result.append(f'Using {el} for resize photo, prediction accuracy of your photo set is: {accuracy_percent} %')
    return result

## Завантаження моделі

In [9]:
model = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_Susana.h5")
model.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_Susana.h5")

In [183]:
x = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'si_rescale'], model=model)

1/1 [==============================] - 0s 27ms/step


C:\Users\kuzik\AppData\Local\Temp\ipykernel_8800\888673498.py:31: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img = skimage.transform.rescale(skimage.transform.resize(img, (for_rescale[1], for_rescale[1])), scale_rating, multichannel=True)


1/1 [==============================] - 0s 27ms/step


In [184]:
for el in x:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 50.0 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 60.0 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 80.0 %
Using si_rescale for resize photo, prediction accuracy of your photo set is: 80.0 %
